## 特殊なフォーマットの読み込み

In [4]:
import imageio

dir_path = "./dlwpt-code-master/data/p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083"
vol_array = imageio.volread(dir_path, 'DICOM')
vol_array.shape

Reading DICOM (examining files): 1/99 files (1.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 99/99  (100.0%)


(99, 512, 512)

In [6]:
import torch
vol = torch.from_numpy(vol_array).float()
vol = torch.unsqueeze(vol, 0)
vol.shape

torch.Size([1, 99, 512, 512])

## 表形式データ読み込みと処理

### 有名なワインデータをテンソルとして読み込む

In [7]:
import csv
import numpy as np
wine_path = "./dlwpt-code-master/data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [8]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [9]:
wineq = torch.from_numpy(wineq_numpy)
wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [10]:
data = wineq[:, :-1] # 最後尾列以外を選択
target = wineq[:, -1] # 最後尾を選択
print(data.shape, target.shape)

torch.Size([4898, 11]) torch.Size([4898])


In [11]:
# target(品質)のone-hotvector化
print(target)
target_long = wineq[:, -1].long()
print(target_long)

target_onehot = torch.zeros(target.shape[0], 10) # 0-9までのラベリング
print(target_onehot)

tensor([6., 6., 6.,  ..., 6., 7., 6.])
tensor([6, 6, 6,  ..., 6, 7, 6])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [12]:
target_onehot.scatter_(1, target_long.unsqueeze(1), 1.0) # カテゴリの数値に対応する列に1を立てる
target_onehot

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [13]:
target_long.unsqueeze(1)

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

In [14]:
# ワインデータ(11列)x4000行くらいの正規化(標準化)
data_mean = torch.mean(data, dim=0)
print("mean: ", data_mean)

data_var = torch.var(data, dim=0)
print("var: ", data_var)

data_normalized = (data - data_mean) / torch.sqrt(data_var)
print("normalized_data: ", data_normalized)

mean:  tensor([6.8548e+00, 2.7824e-01, 3.3419e-01, 6.3914e+00, 4.5772e-02, 3.5308e+01,
        1.3836e+02, 9.9403e-01, 3.1883e+00, 4.8985e-01, 1.0514e+01])
var:  tensor([7.1211e-01, 1.0160e-02, 1.4646e-02, 2.5726e+01, 4.7733e-04, 2.8924e+02,
        1.8061e+03, 8.9455e-06, 2.2801e-02, 1.3025e-02, 1.5144e+00])
normalized_data:  tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])


In [15]:
# 閾値の特定(いらないデータを省く)
bad_indexes = target <= 3 # torch.le(target, 3)でもok
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [16]:
# advanced_indexing機能を使ってTrueに対応する行データ20個をフィルタリング
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [17]:
# 良いワイン、中程度のワイン、悪いワインに分類
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

# 11列のワイン要素毎に良中悪グループごとの平均値を出力する
for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print("{:2} {:20} {:6.2f} {:6.2f} {:6.2f}".format(i, *args))



 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


In [18]:
# 一概には言えないが、6 total sulfur dioxide(二酸化硫黄総量)が閾値として使えそう.
# 大雑把な基準として上記の属性の平均値以下のインデックス(行番号)を見る.
total_sulfur_threshold = 141.83
total_sulfer_data = data[:, 6]
predicted_indexes = total_sulfer_data < total_sulfur_threshold
predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [19]:
# 実際の良いワイン target > 5 のインデックスを見る
actual_indexes = target > 5
actual_indexes.shape, actual_indexes.dtype, actual_indexes.sum()
"""
予測した良品ワインの本数2727本に対して実際の良品ワインの本数は3258本(差約500本)なので、予測は完璧でない
"""

'\n予測した良品ワインの本数2727本に対して実際の良品ワインの本数は3258本(差約500本)なので、予測は完璧でない\n'

In [20]:
# 予測した良品インデックスと実際の良品インデックスの論理和をとって一致度を確認する
n_matches = torch.sum(actual_indexes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item() # 2727
n_actual = torch.sum(actual_indexes).item() # 3258
n_matches, n_matches/n_predicted, n_matches/n_actual
"""
2018本のワインの品質を正確に予測できている. 2727本のワインを高品質と予測していたので、
74%の確立で高品質と予想したワインが実際に高品質であった。
しかし、合計3258本の高品質なワインがある中で、2018本と61%しか特定できていないかった。
"""

'\n2018本のワインの品質を正確に予測できている. 2727本のワインを高品質と予測していたので、\n74%の確立で高品質と予想したワインが実際に高品質であった。\nしかし、合計3258本の高品質なワインがある中で、2018本と61%しか特定できていないかった。\n'

## 時系列データの読み込みと処理

ワシントンD.Cにあるバイクシェアリングシステムのデータ.  
2011年から2012年のレンタル自転車の1時間ごとの利用台数を、天候や季節の情報とともに記録している.  
日次で表データが収集されている状態.

In [21]:
""" 列情報
・レコードのインデックス(行番号) : instant
・日付 : day
・季節 : season(1:春, 2:夏, 3:秋, 4:冬)
・年 : yr(0: 2011, 1: 2012)
・月 : mnth(1~12)
・時間 : hr(0~23)
・休日かどうか : holiday
・曜日 : weekday
・平日かどうか : workingday
・気象状況 : weathersit(1: 晴れ, 2: 霧, 3: 小雨/雪, 4: 本降り/雪)
・温度(℃) : temp
・体感温度(℃) : atemp
・湿度 : hum
・風速 : windspeed
・カジュアルユーザー数: casual
・登録ユーザー数: registered
・使用中レンタル自転車の台数 : cnt
"""
bikes_numpy = np.loadtxt("./dlwpt-code-master/data/p1ch4/bike-sharing-dataset/hour-fixed.csv",
                         dtype=np.float32,
                         delimiter=',',
                         skiprows=1,
                         converters={1: lambda x: float(x[8:10])})
# 1列目の日付文字を曜日に対応する数値に変換

bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [22]:
"""
シート(表)数: N
行数: L
列数 :C
の時(N,C,L)の3次元データになる. 
Lは1データの塊の大きさ.
Lは1h, 1日, 1週など任意で間隔を決める.
"""
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [23]:
# 日, 時間, 各指標を表す17列のデータを3つの軸を持つデータに変形する
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [24]:
# NxCxLの順序になるように転置
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [25]:
# 列属性の変更
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4) # (rowsOfOneday, categoryNum)
first_day[:, 9] # 1時間ごとの天候状況

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

In [26]:
weather_onehot.scatter_(dim=1, index=first_day[:, 9].unsqueeze(1).long() -1, value=1.0) # 1-4 -> 0-3
weather_onehot

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [27]:
torch.cat((bikes[:24], weather_onehot), dim=1)[:1] # 最後尾の(1,0,0,0)がweather_onehot

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

In [28]:
torch.cat((bikes[:24], weather_onehot), dim=1)[0]

tensor([ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
         0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
        16.0000,  1.0000,  0.0000,  0.0000,  0.0000])

## テキストデータの読み込み

In [29]:
# オープンソースのテキストデータを提供するサイト
# Project Genberg(https://www.gutenberg.org)
# English Corpora(https://www.english-corpora.org)

# ジェーン・オースティンの「高慢と偏見」をProject Genbergから読み込む
# (https://www.gutenberg.org/files/1342/1342-0.txt)
with open('./dlwpt-code-master/data/p1ch4/jane-austen/1342-0.txt', encoding='utf-8') as f:
    text = f.read()

### 文字単位でのワンホットエンコーディング

In [30]:
''' 文字単位でのワンホットエンコーディング '''
# 行分割
lines = text.split('\n')
line = lines[200] # 201行目
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [31]:
letter_t = torch.zeros(len(line), 128) # ASCIIコード(128文字)
letter_t.shape

torch.Size([70, 128])

In [34]:
vocabrary = line.lower().split()
vocabrary

['“impossible,',
 'mr.',
 'bennet,',
 'impossible,',
 'when',
 'i',
 'am',
 'not',
 'acquainted',
 'with',
 'him']

In [41]:
# 1を立てるインデックスはエンコーディングしたい文字のインデックスに対応
for vocab in vocabrary:
    for i, letter in enumerate(list(vocab)): # 文字列を文字毎に分割 -> list()
        print(letter)
        letter_index = ord(letter) if ord(letter) < 128 else 0 # <1>
        letter_t[i][letter_index] = 1
letter_t

“
i
m
p
o
s
s
i
b
l
e
,
m
r
.
b
e
n
n
e
t
,
i
m
p
o
s
s
i
b
l
e
,
w
h
e
n
i
a
m
n
o
t
a
c
q
u
a
i
n
t
e
d
w
i
t
h
h
i
m


tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

### 単語レベルでのワンホットエンコーディング

In [44]:
'''単語レベルのワンホットエンコーディング'''

# 1文を単語に分割し、単語を小文字化して句読点を取り除く関数
def clean_words(input_str):
    punctuation = '.,;:"!?“”_-'
    word_list = input_str.lower().replace('\n', '').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [46]:
# 単語とインデックスのマッピングを作成
word_list = sorted(set(clean_words(text)))
word2index_dict = {word : i for (i, word) in enumerate(word_list)}
len(word2index_dict), word2index_dict['impossible']

(15514, 6925)

In [48]:
# エンコード
word_t = torch.zeros(len(words_in_line), len(word2index_dict)) # (vocabNumOfLine, IndexNum)
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))

print(word_t.shape)

 0 6925 impossible
 1 8832 mr
 2 1906 bennet
 3 6925 impossible
 4 14844 when
 5 6769 i
 6  714 am
 7 9198 not
 8  312 acquainted
 9 15085 with
10 6387 him
torch.Size([11, 15514])


### 単語の埋め込み表現

単語を表す空間を例えば100次元空間の浮動小数点で表す. e.g I = (0.1, 0.2, .... 0.221)　→　埋め込み表現(Embedding)  
この際, 意味合いが近い単語同士は100次元空間のベクトル同士も近い距離にあることが理想. 

埋め込み表現は、カテゴリーデータを扱う際にワンホットエンコーディングが面倒になる場面であれば、どんな場面でも有用